In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

In [2]:
cs_ibx_peak = pd.read_csv(r'C:\Users\Gabriel\OneDrive\CUSP\Spring 2022 Classes\ADS\IBX Project\IBXRidership\Network Processing\ctrl_scrs_sta_IBX_peak.csv')
cs_ibx_offpeak = pd.read_csv(r'C:\Users\Gabriel\OneDrive\CUSP\Spring 2022 Classes\ADS\IBX Project\IBXRidership\Network Processing\ctrl_scrs_sta_IBX_offpeak.csv')
cs_noibx_peak = pd.read_csv(r'C:\Users\Gabriel\OneDrive\CUSP\Spring 2022 Classes\ADS\IBX Project\IBXRidership\Network Processing\ctrl_scrs_sta_noIBX_peak.csv')
cs_noibx_offpeak = pd.read_csv(r'C:\Users\Gabriel\OneDrive\CUSP\Spring 2022 Classes\ADS\IBX Project\IBXRidership\Network Processing\ctrl_scrs_sta_noIBX_offpeak.csv')

In [3]:
cs_ibx_peak.rename({'Unnamed: 0':'complex_id'},axis=1,inplace=True)
cs_ibx_offpeak.rename({'Unnamed: 0':'complex_id'},axis=1,inplace=True)
cs_noibx_peak.rename({'Unnamed: 0':'complex_id'},axis=1,inplace=True)
cs_noibx_offpeak.rename({'Unnamed: 0':'complex_id'},axis=1,inplace=True)

In [4]:
#merge the centrality scores together
mg = pd.merge(cs_ibx_peak, cs_ibx_offpeak, on='complex_id',how='left')
mg = pd.merge(mg, cs_noibx_peak, on='complex_id',how='left')
mg = pd.merge(mg, cs_noibx_offpeak, on='complex_id',how='left')
mg.columns = ['Complex_id','cs_ibx_peak','cs_ibx_offpeak','cs_noibx_peak','cs_noibx_offpeak']
mg.head()

,Complex_id,cs_ibx_peak,cs_ibx_offpeak,cs_noibx_peak,cs_noibx_offpeak
0,614c,0.022481,0.013806,0.022649,0.013907
1,611c,0.025420,0.016341,0.025604,0.016466
2,601c,0.024272,0.014947,0.024396,0.015019
3,635c,0.020350,0.012717,0.020454,0.012778
4,620c,0.021319,0.013528,0.021334,0.013548


In [5]:
sts = pd.read_csv(r'C:\Users\Gabriel\OneDrive\CUSP\Spring 2022 Classes\ADS\IBX Project\IBXRidership\Turnstile Data Processing\stations_list (1).csv')
sts.head()

,Unnamed: 0,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Direction Notes,ADA NB,ADA SB,Capital Outage NB,Capital Outage SB
0,118,119,119,L06,BMT,Canarsie,1 AV,M,L,Subway,40.730953,-73.981628,8 Av,Brooklyn,1,NaN,NaN,NaN,NaN,NaN
1,395,395,395,624,IRT,Lexington Av,103 ST,M,6,Subway,40.790600,-73.947478,Uptown & The Bronx,Downtown,0,NaN,NaN,NaN,NaN,NaN
2,155,156,156,A18,IND,8th Av - Fulton St,103 ST,M,B C,Subway,40.796092,-73.961454,Uptown & The Bronx,Downtown & Brooklyn,0,NaN,NaN,NaN,NaN,NaN
3,309,309,309,119,IRT,Broadway - 7Av,103 ST,M,1,Subway,40.799446,-73.968379,Uptown & The Bronx,Downtown,0,NaN,NaN,NaN,NaN,NaN
4,450,450,450,706,IRT,Flushing,103 ST-CORONA PLAZA,Q,7,Elevated,40.749865,-73.862700,Flushing,Manhattan,0,NaN,NaN,NaN,NaN,NaN


In [6]:
sts['Complex ID'] = sts['Complex ID'].astype(str)

In [7]:
#write a new complex_id column. Keep the old one "Complex_id" (with a capital C) for reference
#merge in the complex ID name based on the GTFS Stop ID. This will return NaNs for the actual complex ids, which currently end in the letter "c"
mg = pd.merge(mg, sts[['Complex ID', 'GTFS Stop ID', 'Station ID']],left_on = 'Complex_id', right_on='GTFS Stop ID',how='left', indicator = True)
mg['complex_id'] = np.where(mg._merge == 'left_only',mg.Complex_id,mg['Complex ID'])

#strip the "c" from the remaining complex ids
mg['complex_id'] = mg['complex_id'].str.strip(to_strip='c')
mg.drop(['Complex ID', 'GTFS Stop ID', 'Station ID','_merge'],axis=1,inplace=True)

In [8]:
mg.head()

,Complex_id,cs_ibx_peak,cs_ibx_offpeak,cs_noibx_peak,cs_noibx_offpeak,complex_id
0,614c,0.022481,0.013806,0.022649,0.013907,614
1,611c,0.025420,0.016341,0.025604,0.016466,611
2,601c,0.024272,0.014947,0.024396,0.015019,601
3,635c,0.020350,0.012717,0.020454,0.012778,635
4,620c,0.021319,0.013528,0.021334,0.013548,620


In [9]:
#bring back in the stop name for easy reference later.
sts_nd = sts.drop_duplicates(subset='Complex ID')
mg = pd.merge(mg,sts_nd[['Complex ID', 'Stop Name']],left_on='complex_id',right_on='Complex ID',how='left')
mg.drop(['Complex ID'],axis=1,inplace=True)
mg = mg[['complex_id', 'Stop Name','Complex_id','cs_ibx_peak', 'cs_ibx_offpeak', 'cs_noibx_peak','cs_noibx_offpeak']]

In [10]:

mg.head()

,complex_id,Stop Name,Complex_id,cs_ibx_peak,cs_ibx_offpeak,cs_noibx_peak,cs_noibx_offpeak
0,614,59 ST-COLUMBUS CIRCLE,614c,0.022481,0.013806,0.022649,0.013907
1,611,42 ST-PORT AUTHORITY BUS TERMINAL,611c,0.025420,0.016341,0.025604,0.016466
2,601,14 ST,601c,0.024272,0.014947,0.024396,0.015019
3,635,SOUTH FERRY,635c,0.020350,0.012717,0.020454,0.012778
4,620,BOROUGH HALL,620c,0.021319,0.013528,0.021334,0.013548


In [12]:
mg.shape

(433, 7)

In [11]:
mg.to_csv('complex_centralities.csv')